In [ ]:
import pandas as pd
import numpy as np

# Read the original CSV file
df = pd.read_csv('../../data/accions.csv')

In [ ]:
# Group by 'Sessio' and aggregate the 'Accio' column into a list
grouped_df = df.groupby('Sessio').agg({'Accio': list, 'Tramit': list, 'Usuari': list, 'Data': list}).reset_index()


# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions.csv', index=False)

In [ ]:
grouped_df['Usuari'] = grouped_df['Usuari'].apply(lambda x: x[-1] if isinstance(x, list) and x else x)

In [ ]:
# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions_usuari.csv', index=False)

In [ ]:
# grouped_df = pd.read_csv('../../data/grouped_accions_usuari.csv')
(grouped_df['Tramit'].apply(lambda x: len((x))) == 1).sum()

In [ ]:
filtered_df = grouped_df[~((grouped_df['Usuari'].isna()) & (grouped_df['Tramit'].apply(lambda x: len((x))) == 1))]

In [ ]:
# Write the grouped data to a new CSV file
filtered_df.to_csv('../../data/grouped_accions_filtered.csv', index=False)

In [ ]:
filtered_df = pd.read_csv('../../data/grouped_accions_filtered.csv')

In [ ]:
final_df = filtered_df[~(filtered_df['Tramit'].apply(lambda x: len(eval(x))) == 1)]

In [ ]:
final_df = final_df.drop(columns=['Usuari', 'Accio', 'Data', 'Sessio'])

In [ ]:
final_df.to_csv('../../data/grouped_accions_final.csv', index=False)

In [ ]:
final_df = pd.read_csv('../../data/grouped_accions_final.csv')

#### 


In [ ]:
final_df['Last_Tramit'] = final_df['Tramit'].apply(lambda x: eval(x)[-1] if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df['Tramit'] = final_df['Tramit'].apply(lambda x: str(eval(x)[:-1]) if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df.to_csv('../../data/final_dataset.csv', index=False)

In [ ]:
train_final_df = pd.read_csv('../../data/final_dataset.csv')

In [ ]:
X_train = train_final_df['Tramit']
y_train = train_final_df['Last_Tramit']



In [ ]:
# Crear un dataset de muestra con 50 filas
sample_df = train_final_df.sample(n=50, random_state=42)

# Guardar el dataset de muestra en el directorio
sample_df.to_csv('../../data/sample_train_final_df.csv', index=False)


In [ ]:
# Import the sample_tramits.csv file
sample_tramits_df = pd.read_csv('../../data/sample_tramits.csv')

# Erase the 'Vigent' column
sample_tramits_df = sample_tramits_df.drop(columns=['Vigent'])

# Display the first few rows of the dataframe to verify
print(sample_tramits_df.head())

sample_tramits_df.to_csv('../../data/sample_tramits_df.csv', index=False)

In [ ]:

tramits_df = pd.read_csv('../../data/tramits.csv')


tramits_df['Sequence'] = range(tramits_df.shape[0])


tramits_df.to_csv('../../data/tramits.csv', index=False)

In [ ]:
id_to_sequence = dict(zip(tramits_df['Id'], tramits_df['Sequence']))

final_df['Tramit'] = final_df['Tramit'].apply(lambda x: [id_to_sequence.get(tramit, tramit) for tramit in eval(x)])
final_df['Last_Tramit'] = final_df['Last_Tramit'].apply(lambda x: id_to_sequence.get(x, x))

final_df.to_csv('../../data/final_dataset_mapped.csv', index=False)



In [ ]:

final_df = pd.read_csv('../../data/final_dataset_mapped.csv')
final_df = final_df[final_df['Tramit'].apply(len) <= 30]



In [ ]:
print("Distribución de y_train:", pd.Series(y_train).value_counts())

print("Distribución del 82 en y_train:", (y_train == 314).sum())


In [ ]:
import os
import time
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


# ------------------ Carga y Preprocesamiento del Dataset ------------------
print("### Carga y Preprocesamiento del Dataset ###")
file_path = "../../data/final_dataset_mapped.csv"
data = pd.read_csv(file_path)

# Preprocesar las columnas
data['Tramit'] = data['Tramit'].apply(lambda x: ast.literal_eval(x))  # Convertir strings en listas
if data.isnull().values.any():
    raise ValueError("El archivo CSV contiene valores nulos.")
data['Last_Tramit'] = data['Last_Tramit'].astype(int)

# Preparar las features 'X' y target 'y'
max_seq_len = 20
X_raw = data['Tramit'].tolist()
y_data = data['Last_Tramit'].values
X_data = pad_sequences(X_raw, maxlen=max_seq_len, padding='post', truncating='post')  # Padding de secuencias

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)


# ------------------ Definición de Capas Personalizadas ------------------
print("### Definición de Capas Personalizadas ###")
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
        print(f"Inicializando TokenAndPositionEmbedding con maxlen={maxlen}, vocab_size={vocab_size}, embed_dim={embed_dim}")
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        # Capas Embedding
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        max_len_positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(max_len_positions)  # Generar embeddings posicionales
        tokens = self.token_emb(x)  # Generar embeddings de tokens
        return tokens + positions  # Combinar tokens y posiciones

    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim
        })
        print(f"Serializando capa TokenAndPositionEmbedding con config: {config}")
        return config

    @classmethod
    def from_config(cls, config):
        print(f"Reconstruyendo capa TokenAndPositionEmbedding desde config: {config}")
        return cls(**config)


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        print(f"Inicializando TransformerBlock con embed_dim={embed_dim}, num_heads={num_heads}, ff_dim={ff_dim}, rate={rate}")

        # Guardar configuraciones
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        # Creación de capas
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        })
        print(f"Serializando capa TransformerBlock con config: {config}")
        return config

    @classmethod
    def from_config(cls, config):
        print(f"Reconstruyendo capa TransformerBlock desde config: {config}")
        return cls(**config)

# Resto del código se mantiene igual.
# El modelo utilizará ahora correctamente la clase TokenAndPositionEmbedding.




# ------------------ Definición del Modelo ------------------

print("### Definición del Modelo ###")

def create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate):
    inputs = Input(shape=(max_seq_len,))
    embedding_layer = TokenAndPositionEmbedding(maxlen=max_seq_len, vocab_size=vocab_size, embed_dim=embed_dim)
    x = embedding_layer(inputs)

    for i in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate)(x)
        print(f"Creado TransformerBlock {i + 1} con Dropout Rate = {dropout_rate}")

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(vocab_size, activation="softmax")(x)
    return Model(inputs=inputs, outputs=outputs)


# ------------------ Ejecución Multi-Configuración ------------------
print("### Ejecución Multi-Configuración ###")
configs = pd.DataFrame([  # ~20 min
    # Modelos grandes:,# ~30 min
    {"embed_dim": 512, "num_heads": 16, "ff_dim": 1024, "num_transformer_blocks": 4, "epochs": 8},# ~25-30 min
    {"embed_dim": 256, "num_heads": 8, "ff_dim": 512, "num_transformer_blocks": 3, "epochs": 15}, # ~30+ min
    {"embed_dim": 512, "num_heads": 16, "ff_dim": 1024, "num_transformer_blocks": 4, "epochs": 15},
])

if not os.path.exists('../../models/'):
    os.makedirs('../../models/')

results = []
vocab_size = 502

for index, config in configs.iterrows():
    print(f"\n### Ejecutando Configuración {index + 1}/{len(configs)}: {config.to_dict()}")
    embed_dim = config['embed_dim']
    num_heads = config['num_heads']
    ff_dim = config['ff_dim']
    num_transformer_blocks = config['num_transformer_blocks']
    epochs = config['epochs']
    dropout_rate = 0.2

    # Crear el modelo
    model = create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Entrenar el modelo
    start_time = time.time()
    early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=1
    )
    training_time = time.time() - start_time
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Guardar el modelo
    model_save_path = f"../../models/model_config_{index + 1}.h5"
    try:
        print(f"Guardando modelo en: {model_save_path}")
        model.save(model_save_path)
        print(f"Modelo guardado con éxito en {model_save_path}")
    except Exception as e:
        print(f"Error al guardar el modelo: {e}")
        continue  # Saltar esta configuración si falla

    # Guardar resultados
    results.append({
        "config_index": index,
        "embed_dim": embed_dim,
        "num_heads": num_heads,
        "ff_dim": ff_dim,
        "num_transformer_blocks": num_transformer_blocks,
        "epochs": epochs,
        "train_accuracy": history.history['accuracy'][-1],
        "val_accuracy": max(history.history['val_accuracy']),
        "test_accuracy": test_accuracy,
        "test_loss": test_loss,
        "training_time": training_time,
        "model_path": model_save_path
    })

# Consolidar resultados
if len(results) > 0:  # Comprobar que haya resultados
    try:
        # Convertir lista de diccionarios en DataFrame
        results_df = pd.DataFrame(results)

        # Verificar que el DataFrame no esté vacío
        if results_df.empty:
            print("*** El DataFrame generado a partir de 'results' está vacío. No se guardará ningún archivo.")
        else:
            # Guardar resultados en un archivo CSV
            results_csv_path = '../../data/results.csv'

            # Crear el directorio si no existe
            output_dir = os.path.dirname(results_csv_path)
            if not os.path.exists(output_dir):  # Verificar existencia de la carpeta
                os.makedirs(output_dir)
                print(f"### Directorio creado: {output_dir}")

            # Guardar DataFrame en archivo CSV
            results_df.to_csv(results_csv_path, index=False)
            print(f"### Resultados guardados en el archivo: {results_csv_path}")

    except Exception as e:
        print(f"*** Error al guardar los resultados: {str(e)}")
else:
    print("*** No hay resultados para guardar. La lista 'results' está vacía.")


In [ ]:
## Pruebas de predicciones

# ------------------ Capas Personalizadas ------------------
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim)])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.dropout1(self.att(inputs, inputs, training=training), training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.dropout2(self.ffn(out1, training=training), training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = self.pos_emb(tf.range(start=0, limit=tf.shape(x)[-1]))
        return self.token_emb(x) + positions


# ------------------ Modelo Preentrenado ------------------
def cargar_modelo(ruta_modelo="modelo_transformer_tramites.h5"):
    """Carga el modelo Transformer preentrenado."""
    model = tf.keras.models.load_model(
        ruta_modelo,
        custom_objects={
            'TokenAndPositionEmbedding': TokenAndPositionEmbedding,
            'TransformerBlock': TransformerBlock
        }
    )
    return model


# ------------------ Función de Predicción ------------------
def predecir_tramite(raw_input_sequence, tramits_csv, max_seq_len, model):
    """
    Filtra trámites por 'Vigent=True' y realiza la predicción.
    raw_input_sequence: lista de trámites (secuencia de entrada).
    tramits_csv: Ruta al archivo tramits.csv.
    max_seq_len: Longitud máxima de la secuencia (padding).
    model: Modelo cargado con cargar_modelo.
    """
    tramits_df = pd.read_csv(tramits_csv)
    vigent_indices = tramits_df[tramits_df['Vigent'] == True].index.tolist()

    # Preparar el input
    input_sequence = pad_sequences([raw_input_sequence], maxlen=max_seq_len, padding='post', truncating='post')

    # Predicción
    predictions = model.predict(input_sequence)
    filtered_predictions = {idx: predictions[0][idx] for idx in vigent_indices}
    sorted_predictions = sorted(filtered_predictions.items(), key=lambda x: x[1], reverse=True)

    # Obtener las mejores predicciones
    top_10_predictions = sorted_predictions[:5]
    recommended_tramit, recommended_prob = top_10_predictions[0]

    return recommended_tramit, recommended_prob, top_10_predictions

def call_function(tramit_input):

    # Cargar el modelo preentrenado
    model = cargar_modelo("modelo_transformer_tramites.h5")

    # Realizar una predicción
    raw_input_sequence = tramit_input  # Entrada ejemplo
    tramits_csv = "../../data/tramits.csv"  # Ruta al archivo tramits.csv
    max_seq_len = 20  # Longitud máxima de secuencias

    # Realizar la predicción 
    recommended_tramit, _, top_10_predictions = predecir_tramite(
        raw_input_sequence, tramits_csv, max_seq_len, model
    )
    
    # Filtrar predicciones con probabilidad de al menos 2%
    filtered_predictions = [pred for pred in top_10_predictions if pred[1] >= 0.0001]
    print(filtered_predictions)

    # Leer el archivo tramits.csv para obtener los títulos
    tramits_df = pd.read_csv(tramits_csv)
    
    # Obtener los títulos correspondientes a los índices de las predicciones filtradas
    top_10_titles = [tramits_df.loc[tramits_df['Sequence'] == idx, 'Titol'].values[0] for idx, _ in filtered_predictions]
    
    # Ver resultados
    if filtered_predictions:
        print(f"Trámite recomendado: {tramits_df.loc[tramits_df['Sequence'] == filtered_predictions[0][0], 'Titol'].values[0]}")
        print("\nTop predicciones con al menos 2% de probabilidad:")
        for title in top_10_titles:
            print(title)
    else:
        print("Trámite recomendado: Sol·licitud genèrica")
    
    return top_10_titles if filtered_predictions else ["Sol·licitud genèrica"]
call_function([142, 145, 145, 145, 51, 307, 234, 51, 307, 234, 51, 307, 307, 307])